In [27]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq


In [28]:
path = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/%s/Results.txt'
path_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.csv'
path_res_seq = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.fasta'
experiments = ['mrcv_mites','mrcv_all','sun_all','sun_mites']
path_genome = '/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.dna.toplevel.fa'

In [29]:
#Load shortstack results
dfs = {}
for experiment in experiments:
    experiment_path = path % (experiment,)
    df = pd.read_csv(experiment_path, sep='\t')
    print('Experiment:', experiment)
    print('All results:',len(df.index))
    df = df[df.MIRNA == 'Y']
    print('Only miRNAs:',len(df.index))
    df = df[df.Reads > 20]
    print('Only miRNAs +20 reads:',len(df.index))
    print('*' * 10)
    dfs[experiment] = df

Experiment: mrcv_mites
All results: 1211339
Only miRNAs: 39
Only miRNAs +20 reads: 22
**********
Experiment: mrcv_all
All results: 38532
Only miRNAs: 135
Only miRNAs +20 reads: 135
**********
Experiment: sun_all
All results: 157492
Only miRNAs: 154
Only miRNAs +20 reads: 154
**********
Experiment: sun_mites
All results: 1211339
Only miRNAs: 24
Only miRNAs +20 reads: 12
**********


In [30]:
dfs = [ v for v in dfs.values() ]

In [31]:
df = pd.concat(dfs)

In [32]:
df['seqname'] = df['#Locus'].str.split(':').str[0]
df['position'] = df['#Locus'].str.split(':').str[1]
df['start'] = df.position.str.split('-').str[0].astype(int)
df['end'] = df.position.str.split('-').str[1].astype(int)
df['duplicated'] = 0
df['seqlen'] = df.end - df.start
df['overlap_len'] = df.seqlen  * 0.1
df['overlap_len'] = df.overlap_len.astype(int)
df['start_overlap'] = df.start + df.overlap_len
df['end_overlap'] = df.end - df.overlap_len

In [33]:
print(len(df.index))

323


In [34]:
df[df.index.isin(['876749','876750','876751','27433'])]

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,24,seqname,position,start,end,duplicated,seqlen,overlap_len,start_overlap,end_overlap
876749,6A:23496429-23496595,MITE_T_64052|chr7B|176002823|176003000|AT|187|...,167,111,0.871,18,0.045,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496429-23496595,23496429,23496595,0,166,16,23496445,23496579
876750,6A:23496449-23496574,MITE_T_70309|chr6D|429354379|429354545|TA|23|F...,126,108,0.848,18,0.028,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496449-23496574,23496449,23496574,0,125,12,23496461,23496562
876751,6A:23496458-23496565,MITE_T_95046|chr2B|521974897|521975013|TATA|12...,108,107,0.840,18,0.019,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496458-23496565,23496458,23496565,0,107,10,23496468,23496555
27433,6A:23496419-23496608,Cluster_27434,190,111,0.871,18,0.045,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496419-23496608,23496419,23496608,0,189,18,23496437,23496590


In [35]:
duplicated = []
for k,v in df.iterrows():
    if k in duplicated:
        continue
    other = df[df.index != k]
    other = other[other.seqname == v.seqname]
    other = other[(other.end_overlap >= v.start) & (other.start_overlap <= v.end)]
    if len(other.index) > 0:
        duplicated += other.index.tolist()

171085
[171086]
**********
353559
[11065]
**********
440643
[13327]
**********
526951
[15819]
**********
530300
[15951]
**********
546227
[16699]
**********
634507
[19012]
**********
669360
[82407]
**********
876749
[876750, 876751, 27433]
**********
1018548
[31323]
**********
1088710
[33528, 136099]
**********
1098208
[33899]
**********
871
[2788]
**********
1204
[4364]
**********
4087
[17619]
**********
6423
[26266]
**********
6515
[26660]
**********
8181
[33588]
**********
8687
[36471]
**********
10581
[43056]
**********
11548
[47327]
**********
11747
[47957]
**********
13318
[55260]
**********
15091
[62834]
**********
16106
[66202]
**********
17242
[70524]
**********
17632
[71939]
**********
17676
[72150]
**********
17712
[72277]
**********
17733
[72413]
**********
18433
[76052]
**********
19649
[80021]
**********
20948
[84017]
**********
21290
[85090]
**********
21291
[85090]
**********
22137
[88431]
**********
22941
[92173]
**********
23038
[92528]
**********
23171
[93028]
******

In [40]:
duplicated = set(duplicated)

In [41]:
df = df[~df.index.isin(duplicated)]
print(len(df.index))

253


In [42]:
#get sequences
df.to_csv(path_res, sep='\t', index=None)

In [43]:
fasta_seq = SeqIO.parse(path_genome, 'fasta')
   

In [ ]:
buffer_seqs = []
for record in fasta_seq:
    dff_extract = df[df.seqname == record.id]
    print(record.id, len(dff_extract.index))
    clean_seq = ''.join(str(record.seq).splitlines())
    for k,v in dff_extract.iterrows():
        start = min(v.start,v.end)
        end = max(v.start,v.end)
        new_seq = clean_seq[start:end]
        att = v.Name
        id = att + "_" + record.id + "_" + str(start) + '_' + str(end)
        seq = SeqRecord(Seq(new_seq), id=id, description='')
        buffer_seqs.append(seq)
SeqIO.write(buffer_seqs, path_res_seq, "fasta")

1A 3
1B 7
1D 7
2A 14
